## Unión y depuración de la data

En este notebook uniremos toda la data recopilada anteriormente y crearemos un único DataFrame.  
Posteriormente realizaremos una limpieza de los datos.

In [1]:
import glob
import pandas as pd
from functools import reduce
import numpy as np
pd.options.mode.chained_assignment = None

En lo posterior, necesitamos que el notebook muestre todas las columnas del DataFrame:

In [2]:
pd.set_option('display.max_columns', None)

Unimos todos los csv para crear un único DataFrame:

In [3]:
dfLista = []
ruta = '/home/dsc/Python/Despoblacion/2.Data' 
csv_files = glob.glob(ruta + "/*11.csv")
csv_files.sort() 

for file in csv_files:
    df = pd.read_csv(file, dtype = {'Codigo CCAA' : str, 'Codigo Provincia' : str, 'Codigo Municipio' : str})
    dfLista.append(df) 
    
Despoblacion = reduce(lambda  left,right: pd.merge(left,right,on=['Codigo Municipio'],
                                            how='left'), dfLista)
Despoblacion.head()    

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
0,Pais Vasco,16,Araba/Álava,01,Alegría-Dulantzi,01001,2803.0,1466.0,1337.0,19.95,140.50,568.0,37.0,18.0,14.0,784.0,617,453,164,1992,935,1057,NaN,NaN,NaN,NaN,1.0,4.0,25.0,1.0,1.0,150.0,127.0,277.0,279.0,268.0,547.0,667.0,1346.0,1249.0,645,1965,240,1250,1410,30,90,75,325,1835,255,120,120,230,390,60,60,335,195,2635,195,685,705,1125,105,40,NaN,NaN,NaN,NaN,NaN,655,425,1947.0,1664.0,1843.0,828.73267,1124.34,35.0,6.0
1,Pais Vasco,16,Araba/Álava,01,Amurrio,01002,10014.0,5030.0,4984.0,96.30,103.99,219.0,104.0,37.0,88.0,4397.0,5861,4843,1018,8393,3742,4651,NaN,NaN,NaN,NaN,2.0,9.0,185.0,3.0,NaN,409.0,334.0,742.0,869.0,918.0,1787.0,1316.0,5154.0,4435.0,1285,6935,1795,3975,5120,120,130,675,1750,6975,1955,245,320,745,1285,500,330,955,730,9265,730,2545,2945,2745,690,360,NaN,NaN,NaN,NaN,NaN,2360,1505,6906.0,6196.0,2166.0,3579.79700,5568.50,381.0,367.0
2,Pais Vasco,16,Araba/Álava,01,Añana,01049,168.0,86.0,82.0,21.92,7.66,574.0,1.0,0.0,7.0,30.0,20,14,6,50,16,34,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,6.0,5.0,11.0,12.0,10.0,22.0,181.0,214.0,209.0,15,105,40,65,75,*,*,15,35,102,50,5,10,10,20,10,15,5,30,105,30,45,25,60,5,*,NaN,NaN,NaN,NaN,NaN,15,60,130.0,65.0,229.0,90.70000,1212.00,20.0,9.0
3,Pais Vasco,16,Araba/Álava,01,Aramaio,01003,1496.0,784.0,712.0,73.02,20.49,333.0,13.0,3.0,10.0,224.0,141,88,53,322,177,145,NaN,NaN,NaN,NaN,1.0,2.0,8.0,NaN,NaN,23.0,34.0,57.0,42.0,38.0,80.0,423.0,706.0,681.0,265,955,290,725,680,*,*,95,250,980,325,25,70,45,205,45,70,60,125,1385,125,380,270,545,120,65,NaN,NaN,NaN,NaN,NaN,315,260,1236.0,1014.0,431.0,616.46180,2055.80,191.0,215.0
4,Pais Vasco,16,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,137.0,125,90,35,139,60,79,NaN,NaN,NaN,NaN,NaN,2.0,7.0,NaN,NaN,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40,150,35,100,110,*,*,10,30,160,35,10,15,30,20,5,10,*,20,200,20,45,95,60,5,*,NaN,NaN,NaN,NaN,NaN,65,30,205.0,145.0,0.0,NaN,859.32,19.0,2.0


In [4]:
Despoblacion.shape

(8116, 80)

Transformamos todos los valores del DataFrame en numéricos con dos decimales.  
Antes de ello, en la fase previa de recopilación de la data, se detectó que había tablas con variables que toman el valor \*.
Estos valores corresponden a variables extraídas del INE donde, por secreto estadístico, los registros que tienen un valor inferior a 5, les asigunan un \*.   
Localizamos las columnas que contienen \* y lo sustituimos por la media:

In [5]:
ColumnasAsterisco = Despoblacion.columns[Despoblacion.isin(['*']).any()]
ColumnasAsterisco

Index(['Edad < 16', '16 =< Edad =<64', 'Edad >= 65', 'Soltero/a', 'Casado/a',
       'Separado/a', 'Divorciado/a', 'Viudo/a', 'Nº de Jubilados',
       'Empresario que emplea personal', 'Empresario que no emplea personal',
       '1 hijo/a', '2 hijos/as', '3 hijos/as', '4 o más hijos/as',
       'Nº de Alquileres', 'Hogares Unipersonales', 'Hogares Familiares',
       '1 persona', '2 personas', '3 personas', '4 personas', '5 personas',
       '6 o más personas', 'Nº Hogares si Internet', 'Nº Hogares no Internet'],
      dtype='object')

In [6]:
Despoblacion[ColumnasAsterisco] = Despoblacion[ColumnasAsterisco].replace('*', 2)

Ahora si que ya podemos realizar el cambio de formato desde la columna número 4 en adelante:

In [7]:
Columnas = Despoblacion.columns[6 : ]
Despoblacion[Columnas] = Despoblacion[Columnas].astype(float).round(2)

In [8]:
Formato = Despoblacion.dtypes
print(Formato.to_string())

Nombre CCAA                                 object
Codigo CCAA                                 object
Nombre Provincia                            object
Codigo Provincia                            object
Nombre Municipio                            object
Codigo Municipio                            object
Poblacion Total                            float64
Poblacion Hombre                           float64
Poblacion Mujer                            float64
Superficie Km2                             float64
Densidad Poblacion                         float64
Altitud Municipios                         float64
Nacimientos                                float64
Matrimonios                                float64
Defunciones                                float64
Total Regimen SS                           float64
Total Contratos                            float64
Contratos Hombres                          float64
Contratos Mujeres                          float64
Total Paro Registrado          

In [9]:
Despoblacion.head()

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
0,Pais Vasco,16,Araba/Álava,01,Alegría-Dulantzi,01001,2803.0,1466.0,1337.0,19.95,140.50,568.0,37.0,18.0,14.0,784.0,617.0,453.0,164.0,1992.0,935.0,1057.0,NaN,NaN,NaN,NaN,1.0,4.0,25.0,1.0,1.0,150.0,127.0,277.0,279.0,268.0,547.0,667.0,1346.0,1249.0,645.0,1965.0,240.0,1250.0,1410.0,30.0,90.0,75.0,325.0,1835.0,255.0,120.0,120.0,230.0,390.0,60.0,60.0,335.0,195.0,2635.0,195.0,685.0,705.0,1125.0,105.0,40.0,NaN,NaN,NaN,NaN,NaN,655.0,425.0,1947.0,1664.0,1843.0,828.73,1124.34,35.0,6.0
1,Pais Vasco,16,Araba/Álava,01,Amurrio,01002,10014.0,5030.0,4984.0,96.30,103.99,219.0,104.0,37.0,88.0,4397.0,5861.0,4843.0,1018.0,8393.0,3742.0,4651.0,NaN,NaN,NaN,NaN,2.0,9.0,185.0,3.0,NaN,409.0,334.0,742.0,869.0,918.0,1787.0,1316.0,5154.0,4435.0,1285.0,6935.0,1795.0,3975.0,5120.0,120.0,130.0,675.0,1750.0,6975.0,1955.0,245.0,320.0,745.0,1285.0,500.0,330.0,955.0,730.0,9265.0,730.0,2545.0,2945.0,2745.0,690.0,360.0,NaN,NaN,NaN,NaN,NaN,2360.0,1505.0,6906.0,6196.0,2166.0,3579.80,5568.50,381.0,367.0
2,Pais Vasco,16,Araba/Álava,01,Añana,01049,168.0,86.0,82.0,21.92,7.66,574.0,1.0,0.0,7.0,30.0,20.0,14.0,6.0,50.0,16.0,34.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,6.0,5.0,11.0,12.0,10.0,22.0,181.0,214.0,209.0,15.0,105.0,40.0,65.0,75.0,2.0,2.0,15.0,35.0,102.0,50.0,5.0,10.0,10.0,20.0,10.0,15.0,5.0,30.0,105.0,30.0,45.0,25.0,60.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,15.0,60.0,130.0,65.0,229.0,90.70,1212.00,20.0,9.0
3,Pais Vasco,16,Araba/Álava,01,Aramaio,01003,1496.0,784.0,712.0,73.02,20.49,333.0,13.0,3.0,10.0,224.0,141.0,88.0,53.0,322.0,177.0,145.0,NaN,NaN,NaN,NaN,1.0,2.0,8.0,NaN,NaN,23.0,34.0,57.0,42.0,38.0,80.0,423.0,706.0,681.0,265.0,955.0,290.0,725.0,680.0,2.0,2.0,95.0,250.0,980.0,325.0,25.0,70.0,45.0,205.0,45.0,70.0,60.0,125.0,1385.0,125.0,380.0,270.0,545.0,120.0,65.0,NaN,NaN,NaN,NaN,NaN,315.0,260.0,1236.0,1014.0,431.0,616.46,2055.80,191.0,215.0
4,Pais Vasco,16,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,137.0,125.0,90.0,35.0,139.0,60.0,79.0,NaN,NaN,NaN,NaN,NaN,2.0,7.0,NaN,NaN,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40.0,150.0,35.0,100.0,110.0,2.0,2.0,10.0,30.0,160.0,35.0,10.0,15.0,30.0,20.0,5.0,10.0,2.0,20.0,200.0,20.0,45.0,95.0,60.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,65.0,30.0,205.0,145.0,0.0,NaN,859.32,19.0,2.0


La dimensión del DataFrame es de 8.116 filas coincidiendo con el número de municipios en España en 2011.  
No obstante, veamos si hay algún Codigo Municipio duplicado:

In [10]:
Despoblacion[Despoblacion['Codigo Municipio'].duplicated()]

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas


Analizamos los valores nulos:

In [11]:
Valores = Despoblacion.isnull().sum()
print(Valores.to_string())

Nombre CCAA                                   0
Codigo CCAA                                   0
Nombre Provincia                              0
Codigo Provincia                              0
Nombre Municipio                              0
Codigo Municipio                              0
Poblacion Total                               0
Poblacion Hombre                              0
Poblacion Mujer                               0
Superficie Km2                                0
Densidad Poblacion                            0
Altitud Municipios                            0
Nacimientos                                   0
Matrimonios                                   0
Defunciones                                   0
Total Regimen SS                             36
Total Contratos                               0
Contratos Hombres                             0
Contratos Mujeres                             0
Total Paro Registrado                         0
Parados Hombres                         

**Total Regimen SS**: El campo contiene 36 valores nulos.  
La tabla de afiliados no contiene los 36 municipios correspondientes:

In [12]:
AfiliadosNulos = Despoblacion[Despoblacion['Total Regimen SS'].isnull()]
AfiliadosNulos = AfiliadosNulos['Codigo Municipio'].values.tolist()
AfiliadosNulos

['02901',
 '06902',
 '08904',
 '08903',
 '09102',
 '09478',
 '13903',
 '19031',
 '19116',
 '19171',
 '19182',
 '20907',
 '25912',
 '25913',
 '26035',
 '26077',
 '26181',
 '29902',
 '31119',
 '34110',
 '36901',
 '41903',
 '43907',
 '44142',
 '44203',
 '44204',
 '44211',
 '44218',
 '46902',
 '46903',
 '48914',
 '48915',
 '50042',
 '50047',
 '50080',
 '50902']

In [13]:
Afiliaciones11 = pd.read_csv('/home/dsc/Python/Despoblacion/2.Data/06_Output_Afiliados_SS_11.csv' ,
                             dtype = {'Codigo Municipio' : str})

Afiliaciones11[Afiliaciones11['Codigo Municipio'].isin(AfiliadosNulos)]

,Codigo Municipio,Total Regimen SS


Eliminamos los 36 registros:

In [14]:
Despoblacion = Despoblacion.dropna(subset = ['Total Regimen SS'])

**Número de transacciones anuales 2011**: Casualmente son 4 municipios que posteriormente, en 2016, se fusionaron: Cercedo-Cotobade, Oza-Cesuras. Vemos si en el DataFrame de transacciones aparecen los municipios fusionados:


In [15]:
Despoblacion[Despoblacion['Número de transacciones anuales 2011'].isnull()].head()

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
2145,Galicia,12,"Coruña, A",15,Cesuras,15026,2225.0,1090.0,1135.0,79.46,28.00,231.0,3.0,9.0,35.0,325.0,67.0,40.0,27.0,1396.0,940.0,456.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,21.0,22.0,43.0,25.0,23.0,48.0,1410.0,1417.0,1402.0,2.0,1340.0,495.0,870.0,995.0,2.0,2.0,310.0,225.0,1027.0,520.0,2.0,2.0,2.0,570.0,2.0,2.0,2.0,2.0,2090.0,2.0,105.0,455.0,145.0,1455.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,355.0,NaN,NaN,0.0,307.00,3005.72,558.0,642.0
2182,Galicia,12,"Coruña, A",15,Oza dos Ríos,15063,3229.0,1622.0,1607.0,72.14,44.76,231.0,23.0,11.0,45.0,656.0,392.0,286.0,106.0,2512.0,1195.0,1317.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,66.0,61.0,127.0,65.0,52.0,117.0,1857.0,2084.0,2043.0,335.0,1855.0,1030.0,1175.0,1585.0,2.0,80.0,335.0,1130.0,1650.0,1180.0,195.0,240.0,305.0,470.0,170.0,70.0,2.0,285.0,2785.0,285.0,915.0,1125.0,585.0,125.0,185.0,NaN,NaN,NaN,NaN,NaN,525.0,795.0,NaN,NaN,177.0,390.00,1759.16,428.0,535.0
5280,Galicia,12,Pontevedra,36,Cerdedo,36011,2297.0,1063.0,1234.0,134.70,17.05,374.0,7.0,3.0,32.0,234.0,154.0,64.0,90.0,1319.0,670.0,649.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,75.0,55.0,130.0,36.0,28.0,64.0,1108.0,1192.0,1189.0,130.0,1010.0,710.0,540.0,890.0,2.0,90.0,315.0,715.0,955.0,750.0,25.0,60.0,250.0,205.0,140.0,75.0,85.0,305.0,1540.0,305.0,460.0,655.0,345.0,85.0,2.0,NaN,NaN,NaN,NaN,NaN,230.0,630.0,1162.0,718.0,168.0,296.56,725.01,192.0,158.0
5281,Galicia,12,Pontevedra,36,Cotobade,36012,4432.0,2101.0,2331.0,79.84,55.51,374.0,26.0,12.0,71.0,605.0,272.0,170.0,102.0,4957.0,2628.0,2329.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,91.0,107.0,198.0,95.0,70.0,165.0,2544.0,2565.0,2561.0,525.0,2580.0,1295.0,1720.0,2245.0,2.0,2.0,390.0,1840.0,2010.0,1320.0,2.0,270.0,195.0,435.0,205.0,245.0,295.0,340.0,3880.0,340.0,900.0,915.0,595.0,445.0,1200.0,NaN,NaN,NaN,NaN,NaN,540.0,995.0,3721.0,2622.0,0.0,520.00,992.45,402.0,463.0


In [16]:
Transacciones11 = pd.read_csv('/home/dsc/Python/Despoblacion/2.Data/12_Output_Transacciones_Inmobiliarias_11.csv' ,
                              dtype = {'Codigo Provincia' : str, 'Codigo Municipio' : str})

In [17]:
Transacciones11[Transacciones11['Codigo Municipio'] == '36902']#Cercedo-Cotobade

,Codigo Municipio,Número de transacciones anuales 2011
5344,36902,NaN


In [18]:
Transacciones11[Transacciones11['Codigo Municipio'] == '15902'] #Oza-Cesuras

,Codigo Municipio,Número de transacciones anuales 2011
2218,15902,27.0


Eliminamos los registros de Cercedo y Cotobade.   
Tenemos el dato global de Oza-Cesuras, estimamos: la población es un 60/40, asignamos 16/11:

In [19]:
Despoblacion.loc[2145, 'Número de transacciones anuales 2011'] = 11
Despoblacion.loc[2183, 'Número de transacciones anuales 2011'] = 16

Despoblacion = Despoblacion.dropna(subset = ['Número de transacciones anuales 2011'])

**Nº Establecimientos Agricultura, Industria , Construccion , Servicios, Centros Sanitarios, Centros Educativos, Residencias Mayores, Instituciones Religiosas y Militares**: Los valores nulos corresponden a la falta de dichos centros en esos municipios.   
Los sustituimos por ceros:


In [20]:
ColumnasNan = ['Nº Establecimientos Agricultura', 'Nº Establecimientos Industria', 'Nº Establecimientos Construccion',
               'Nº Establecimientos Servicios', 'Nº Centros Sanitarios', 'Nº Centros Educativos', 
               'Nº Residencias Mayores', 'Nº Instituciones Religiosas y Militares']

Despoblacion[ColumnasNan] = Despoblacion[ColumnasNan].replace(np.nan, 0)

**Nº de extranjeros por municipio**: hay 772 nulos en hombres, 730 en mujeres y 466 en el total.
Además, si nos fijamos en la población total de estos municipios, esta está muy por debajo de la media.  
Entendemos los valores nulos como cero casos muestrales.

In [21]:
round(Despoblacion['Poblacion Total'].mean(), 2)

5834.07

In [22]:
NulosExtranjeros = Despoblacion[Despoblacion['Nº Total Extranjeros'].isnull()]
(NulosExtranjeros['Poblacion Total'] < 5834.07).sum()

466

In [23]:
ColumnasNan = ['Nº Hombres Extranjeros', 'Nº Mujeres Extranjeras', 'Nº Total Extranjeros']
Despoblacion[ColumnasNan] = Despoblacion[ColumnasNan].replace(np.nan, 0)

**Nº de personas otras comunidades autónomas**: Nº Hombres Otra CCAA hay 211 nulos, Nº Mujeres Otra CCAA hay 280 y en el total hay 96 nulos.  
Entendemos los valores nulos como cero casos muestrales.

In [24]:
ColumnasNan = ['Nº Hombres Otra CCAA', 'Nº Mujeres Otra CCAA', 'Nº Total Otra CCAA']
Despoblacion[ColumnasNan] = Despoblacion[ColumnasNan].replace(np.nan, 0)

**Nº de vehiculos**: 8 de los 9 valores nulos coinciden con que también son nulos los valores de la variable número de conductores.  
Asignamos el valor cero en ambos campos:

In [25]:
Despoblacion[Despoblacion['Nº Vehiculos'].isnull()]

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
749,Extremadura,11,Badajoz,06,Sancti-Spíritus,06118,243.0,131.0,112.0,33.56,7.24,485.0,1.0,0.0,7.0,33.0,10.0,5.0,5.0,277.0,160.0,117.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,16.0,14.0,30.0,250.0,262.0,261.0,20.0,125.0,90.0,75.0,125.0,2.0,2.0,30.0,115.0,92.0,100.0,2.0,20.0,20.0,30.0,15.0,15.0,2.0,40.0,195.0,40.0,90.0,60.0,25.0,10.0,10.0,46.0,127.0,235.0,4.0,2.0,20.0,90.0,NaN,2.0,0.0,146.30,1261.50,52.0,20.0
2145,Galicia,12,"Coruña, A",15,Cesuras,15026,2225.0,1090.0,1135.0,79.46,28.00,231.0,3.0,9.0,35.0,325.0,67.0,40.0,27.0,1396.0,940.0,456.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,0.0,0.0,21.0,22.0,43.0,25.0,23.0,48.0,1410.0,1417.0,1402.0,2.0,1340.0,495.0,870.0,995.0,2.0,2.0,310.0,225.0,1027.0,520.0,2.0,2.0,2.0,570.0,2.0,2.0,2.0,2.0,2090.0,2.0,105.0,455.0,145.0,1455.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,355.0,NaN,NaN,0.0,307.00,3005.72,558.0,642.0
4678,Navarra,15,Navarra,31,Castejón,31070,4316.0,2175.0,2141.0,18.07,238.85,277.0,51.0,16.0,41.0,1151.0,947.0,519.0,428.0,5098.0,3081.0,2017.0,3.0,21.0,12.0,65.0,1.0,4.0,21.0,1.0,0.0,476.0,369.0,844.0,514.0,576.0,1090.0,1294.0,2152.0,2071.0,755.0,2650.0,750.0,1650.0,2120.0,70.0,65.0,245.0,1125.0,2255.0,795.0,90.0,165.0,280.0,555.0,225.0,95.0,725.0,305.0,3785.0,305.0,905.0,815.0,1240.0,825.0,65.0,NaN,NaN,NaN,NaN,NaN,710.0,805.0,NaN,NaN,10212.0,3303.00,1103.35,26.0,7.0
4824,Navarra,15,Navarra,31,Sada,31212,195.0,103.0,92.0,12.65,15.42,510.0,1.0,0.0,4.0,53.0,31.0,21.0,10.0,30.0,29.0,1.0,0.0,2.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,4.0,3.0,7.0,1.0,3.0,4.0,112.0,117.0,117.0,10.0,95.0,80.0,80.0,85.0,2.0,2.0,20.0,65.0,105.0,80.0,10.0,20.0,2.0,20.0,5.0,25.0,2.0,20.0,155.0,20.0,60.0,55.0,45.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,30.0,50.0,NaN,NaN,14.0,67.00,1403.49,40.0,1.0
7266,Castilla y León,07,Valladolid,47,"Campillo, El",47031,231.0,129.0,102.0,32.80,7.04,744.0,2.0,1.0,3.0,28.0,24.0,24.0,0.0,129.0,56.0,73.0,4.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,7.0,7.0,14.0,9.0,8.0,17.0,224.0,229.0,229.0,20.0,110.0,100.0,80.0,125.0,2.0,2.0,15.0,65.0,125.0,105.0,15.0,10.0,2.0,25.0,10.0,20.0,2.0,15.0,215.0,15.0,80.0,50.0,55.0,20.0,5.0,44.0,68.0,199.0,13.0,3.0,20.0,70.0,NaN,NaN,0.0,82.99,1937.63,55.0,10.0
7448,Castilla y León,07,Valladolid,47,Villanueva de los Infantes,47221,123.0,70.0,53.0,19.17,6.42,745.0,1.0,0.0,0.0,25.0,11.0,9.0,2.0,101.0,73.0,28.0,5.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,2.0,7.0,74.0,76.0,76.0,2.0,85.0,25.0,55.0,55.0,2.0,2.0,5.0,20.0,77.0,25.0,2.0,10.0,2.0,5.0,10.0,2.0,2.0,15.0,100.0,15.0,40.0,45.0,20.0,2.0,2.0,53.0,23.0,81.0,4.0,2.0,15.0,40.0,NaN,NaN,0.0,0.00,1380.82,18.0,3.0
7460,Castilla y

In [26]:
Despoblacion[['Nº Vehiculos', 'Nº Conductores']] \
.loc[((Despoblacion['Nº Vehiculos'].isnull()) & (Despoblacion['Nº Conductores'].isnull()))] = 0

Queda un municipio con el número de vehículos cero y 2 conductores. Como la población es muy pequeña, 230 habitantes, asignamos el valor 0:

In [27]:
Despoblacion['Nº Vehiculos'] = Despoblacion['Nº Vehiculos'].replace(np.nan, 0)

**Nº condutores**: Hay 3 municipios con valores nulos y con nº de vehículos: 579, 2 y 6. En estos dos últimos asumimos que el número de conductores es cero. Para el municipio de 579 vehículos se ha buscado y encontrado el nº de conductores en 2012: 351. Sustituimos el nulo por dicho valor.  
Para los nulos con cero número de vehículos, sustituimos el valor por cero.

In [28]:
Despoblacion[Despoblacion['Nº Conductores'].isnull()]

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
2145,Galicia,12,"Coruña, A",15,Cesuras,15026,2225.0,1090.0,1135.0,79.46,28.00,231.0,3.0,9.0,35.0,325.0,67.0,40.0,27.0,1396.0,940.0,456.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,0.0,0.0,21.0,22.0,43.0,25.0,23.0,48.0,1410.0,1417.0,1402.0,2.0,1340.0,495.0,870.0,995.0,2.0,2.0,310.0,225.0,1027.0,520.0,2.0,2.0,2.0,570.0,2.0,2.0,2.0,2.0,2090.0,2.0,105.0,455.0,145.0,1455.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,355.0,0.0,NaN,0.00,307.00,3005.72,558.0,642.0
4626,Navarra,15,Navarra,31,Arantza,31022,644.0,354.0,290.0,31.71,20.31,270.0,9.0,2.0,6.0,97.0,54.0,30.0,24.0,278.0,132.0,146.0,0.0,1.0,3.0,10.0,1.0,2.0,0.0,0.0,0.0,2.0,8.0,10.0,34.0,21.0,55.0,247.0,346.0,343.0,90.0,395.0,160.0,305.0,285.0,2.0,2.0,45.0,205.0,345.0,170.0,2.0,25.0,30.0,95.0,35.0,20.0,2.0,65.0,580.0,65.0,110.0,145.0,230.0,70.0,25.0,NaN,NaN,NaN,NaN,NaN,95.0,150.0,579.0,NaN,453.00,135.53,1786.42,126.0,224.0
4678,Navarra,15,Navarra,31,Castejón,31070,4316.0,2175.0,2141.0,18.07,238.85,277.0,51.0,16.0,41.0,1151.0,947.0,519.0,428.0,5098.0,3081.0,2017.0,3.0,21.0,12.0,65.0,1.0,4.0,21.0,1.0,0.0,476.0,369.0,844.0,514.0,576.0,1090.0,1294.0,2152.0,2071.0,755.0,2650.0,750.0,1650.0,2120.0,70.0,65.0,245.0,1125.0,2255.0,795.0,90.0,165.0,280.0,555.0,225.0,95.0,725.0,305.0,3785.0,305.0,905.0,815.0,1240.0,825.0,65.0,NaN,NaN,NaN,NaN,NaN,710.0,805.0,0.0,NaN,10212.00,3303.00,1103.35,26.0,7.0
4824,Navarra,15,Navarra,31,Sada,31212,195.0,103.0,92.0,12.65,15.42,510.0,1.0,0.0,4.0,53.0,31.0,21.0,10.0,30.0,29.0,1.0,0.0,2.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,4.0,3.0,7.0,1.0,3.0,4.0,112.0,117.0,117.0,10.0,95.0,80.0,80.0,85.0,2.0,2.0,20.0,65.0,105.0,80.0,10.0,20.0,2.0,20.0,5.0,25.0,2.0,20.0,155.0,20.0,60.0,55.0,45.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,30.0,50.0,0.0,NaN,14.00,67.00,1403.49,40.0,1.0
6286,Castilla y León,07,Soria,42,Rebollar,42151,44.0,22.0,22.0,10.38,4.24,1134.0,0.0,0.0,2.0,7.0,3.0,1.0,2.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,33.0,36.0,36.0,2.0,10.0,25.0,15.0,20.0,2.0,2.0,2.0,2.0,27.0,20.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,10.0,20.0,10.0,10.0,2.0,10.0,2.0,5.0,25.0,53.0,25.0,2.0,1.0,2.0,15.0,2.0,NaN,0.00,4.48,860.34,9.0,5.0
6433,Cataluña,09,Tarragona,43,"Molar, El",43085,305.0,166.0,139.0,22.91,13.31,209.0,2.0,0.0,5.0,61.0,19.0,9.0,10.0,128.0,74.0,54.0,4.0,3.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,21.0,12.0,33.0,7.0,7.0,14.0,224.0,238.0,238.0,30.0,175.0,90.0,110.0,150.0,2.0,10.0,20.0,70.0,185.0,100.0,2.0,25.0,10.0,45.0,10.0,10.0,10.0,50.0,240.0,50.0,55.0,55.0,80.0,45.0,10.0,48.0,56.0,204.0,6.0,1.0,65.0,60.0,6.0,NaN,87.48,97.00,897.42,127.0,2.0
7266,Castilla y León,07,Valladolid,47,"Campillo, E

In [29]:
Despoblacion.loc[4626, 'Nº Conductores'] = 351
Despoblacion['Nº Conductores'] = Despoblacion['Nº Conductores'].replace(np.nan, 0)

**Total impuestos (Miles de €)**: Son 5 registros con valores nulos. Buscando estos valores en el ministerio de hacienda, el valor que les asignan es cero. Sustituimos los nulos por dicho valor. 

In [30]:
Despoblacion[Despoblacion['Total impuestos (Miles de €)'].isnull()]

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
4,Pais Vasco,16,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,137.0,125.0,90.0,35.0,139.0,60.0,79.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40.0,150.0,35.0,100.0,110.0,2.0,2.0,10.0,30.0,160.0,35.0,10.0,15.0,30.0,20.0,5.0,10.0,2.0,20.0,200.0,20.0,45.0,95.0,60.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,65.0,30.0,205.0,145.0,0.0,NaN,859.32,19.0,2.0
6541,Aragón,02,Teruel,44,Alcalá de la Selva,44012,516.0,274.0,242.0,104.95,4.92,1403.0,2.0,1.0,2.0,170.0,122.0,66.0,56.0,362.0,195.0,167.0,0.0,1.0,5.0,29.0,1.0,1.0,27.0,0.0,0.0,33.0,31.0,64.0,64.0,59.0,123.0,751.0,1797.0,1795.0,70.0,320.0,110.0,255.0,195.0,2.0,2.0,40.0,115.0,305.0,115.0,45.0,35.0,40.0,55.0,25.0,15.0,75.0,45.0,420.0,45.0,80.0,170.0,145.0,55.0,2.0,30.0,463.0,1606.0,31.0,10.0,80.0,110.0,430.0,291.0,315.0,NaN,6588.94,61.0,24.0
6545,Aragón,02,Teruel,44,Aliaga,44017,382.0,214.0,168.0,193.08,1.98,1105.0,2.0,0.0,14.0,96.0,40.0,17.0,23.0,146.0,60.0,86.0,3.0,3.0,3.0,10.0,3.0,1.0,3.0,0.0,0.0,13.0,14.0,27.0,16.0,14.0,30.0,526.0,650.0,649.0,25.0,230.0,100.0,155.0,165.0,2.0,2.0,30.0,85.0,215.0,120.0,20.0,25.0,15.0,45.0,20.0,10.0,60.0,65.0,280.0,65.0,125.0,75.0,45.0,50.0,2.0,43.0,219.0,522.0,14.0,5.0,80.0,95.0,401.0,234.0,0.0,NaN,15374.04,78.0,52.0
7834,Aragón,02,Zaragoza,50,Alcalá de Moncayo,50014,205.0,118.0,87.0,13.76,14.90,763.0,2.0,0.0,1.0,22.0,23.0,13.0,10.0,158.0,81.0,77.0,0.0,0.0,3.0,3.0,1.0,0.0,2.0,0.0,0.0,14.0,9.0,22.0,8.0,10.0,18.0,182.0,190.0,190.0,25.0,110.0,60.0,80.0,85.0,2.0,2.0,20.0,60.0,80.0,55.0,2.0,15.0,2.0,30.0,2.0,10.0,25.0,25.0,165.0,25.0,65.0,40.0,50.0,10.0,2.0,65.0,122.0,173.0,3.0,0.0,35.0,55.0,210.0,94.0,256.0,NaN,493.64,52.0,3.0
7835,Aragón,02,Zaragoza,50,Alconchel de Ariza,50015,96.0,55.0,41.0,34.85,2.75,893.0,0.0,0.0,1.0,21.0,11.0,6.0,5.0,18.0,12.0,6.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,209.0,212.0,212.0,2.0,40.0,45.0,35.0,40.0,2.0,2.0,15.0,45.0,22.0,45.0,2.0,15.0,2.0,15.0,2.0,2.0,2.0,10.0,65.0,10.0,40.0,30.0,2.0,2.0,2.0,65.0,250.0,186.0,6.0,1.0,2.0,35.0,110.0,57.0,0.0,NaN,3389.57,27.0,11.0


In [31]:
Despoblacion['Total impuestos (Miles de €)'] = Despoblacion['Total impuestos (Miles de €)'].replace(np.nan, 0)

**Superficie (Ha) Agricolas, Nº Explotaciones Agrícolas, Nº Explotaciones Ganaderas**: Los nulos son municipios en los que no hay actividad.  
Sustituimos los nulos por ceros.

In [32]:
Despoblacion[['Superficie (Ha) Agricolas', 'Nº Explotaciones Agrícolas', 'Nº Explotaciones Ganaderas' ]] = \
Despoblacion[['Superficie (Ha) Agricolas', 'Nº Explotaciones Agrícolas', 'Nº Explotaciones Ganaderas' ]].\
replace(np.nan, 0)

**Edad Media Edificios y número de establecimientos**: Hay 522 y 517 valores nulos respectivamente. 
En este caso son registros nulos de los que no disponemos de sus valores.  
Si decidimos eliminarlos tendremos el siguiente inconveniente:

In [33]:
Nulos = Despoblacion[Despoblacion['Edad Media Edificios'].isnull()]
Nulos['Nombre CCAA'].unique()

array(['Pais Vasco', 'Castilla y León', 'Galicia', 'Aragón', 'Navarra'],
      dtype=object)

In [34]:
print(Nulos['Codigo CCAA'].value_counts()['16']) #Codigo CCAA Pais Vasco
print(Nulos['Codigo CCAA'].value_counts()['15']) #Codigo CCAA Navarra

248
271


Pais Vasco y Navarra tienen 251 y 272 municipios respectivamente. Si eliminamos los nulos de Edad Media Edificios y número de establecimientos, nos quedamos prácticamente sin registros de estas dos CCAA.  
Por tanto, se decide eliminar estas dos variables del DataFrame.

In [35]:
Columnas = ['Edad Media Edificios', 'Uso Industrial', 'Uso Residelcial', 'Uso Terciario', 'Otros Usos']
Despoblacion = Despoblacion.drop(Columnas, axis = 1)

Eliminados todos los nulos:

In [36]:
Despoblacion.isnull().sum().sum()

0

DataFrame definitivo:

In [37]:
Despoblacion.head()

,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
0,Pais Vasco,16,Araba/Álava,01,Alegría-Dulantzi,01001,2803.0,1466.0,1337.0,19.95,140.50,568.0,37.0,18.0,14.0,784.0,617.0,453.0,164.0,1992.0,935.0,1057.0,0.0,0.0,0.0,0.0,1.0,4.0,25.0,1.0,1.0,150.0,127.0,277.0,279.0,268.0,547.0,667.0,1346.0,1249.0,645.0,1965.0,240.0,1250.0,1410.0,30.0,90.0,75.0,325.0,1835.0,255.0,120.0,120.0,230.0,390.0,60.0,60.0,335.0,195.0,2635.0,195.0,685.0,705.0,1125.0,105.0,40.0,655.0,425.0,1947.0,1664.0,1843.0,828.73,1124.34,35.0,6.0
1,Pais Vasco,16,Araba/Álava,01,Amurrio,01002,10014.0,5030.0,4984.0,96.30,103.99,219.0,104.0,37.0,88.0,4397.0,5861.0,4843.0,1018.0,8393.0,3742.0,4651.0,0.0,0.0,0.0,0.0,2.0,9.0,185.0,3.0,0.0,409.0,334.0,742.0,869.0,918.0,1787.0,1316.0,5154.0,4435.0,1285.0,6935.0,1795.0,3975.0,5120.0,120.0,130.0,675.0,1750.0,6975.0,1955.0,245.0,320.0,745.0,1285.0,500.0,330.0,955.0,730.0,9265.0,730.0,2545.0,2945.0,2745.0,690.0,360.0,2360.0,1505.0,6906.0,6196.0,2166.0,3579.80,5568.50,381.0,367.0
2,Pais Vasco,16,Araba/Álava,01,Añana,01049,168.0,86.0,82.0,21.92,7.66,574.0,1.0,0.0,7.0,30.0,20.0,14.0,6.0,50.0,16.0,34.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,6.0,5.0,11.0,12.0,10.0,22.0,181.0,214.0,209.0,15.0,105.0,40.0,65.0,75.0,2.0,2.0,15.0,35.0,102.0,50.0,5.0,10.0,10.0,20.0,10.0,15.0,5.0,30.0,105.0,30.0,45.0,25.0,60.0,5.0,2.0,15.0,60.0,130.0,65.0,229.0,90.70,1212.00,20.0,9.0
3,Pais Vasco,16,Araba/Álava,01,Aramaio,01003,1496.0,784.0,712.0,73.02,20.49,333.0,13.0,3.0,10.0,224.0,141.0,88.0,53.0,322.0,177.0,145.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,0.0,0.0,23.0,34.0,57.0,42.0,38.0,80.0,423.0,706.0,681.0,265.0,955.0,290.0,725.0,680.0,2.0,2.0,95.0,250.0,980.0,325.0,25.0,70.0,45.0,205.0,45.0,70.0,60.0,125.0,1385.0,125.0,380.0,270.0,545.0,120.0,65.0,315.0,260.0,1236.0,1014.0,431.0,616.46,2055.80,191.0,215.0
4,Pais Vasco,16,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,137.0,125.0,90.0,35.0,139.0,60.0,79.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40.0,150.0,35.0,100.0,110.0,2.0,2.0,10.0,30.0,160.0,35.0,10.0,15.0,30.0,20.0,5.0,10.0,2.0,20.0,200.0,20.0,45.0,95.0,60.0,5.0,2.0,65.0,30.0,205.0,145.0,0.0,0.00,859.32,19.0,2.0


In [38]:
Despoblacion.shape

(8077, 75)

In [39]:
Despoblacion.to_csv('Despoblacion_11.csv', header = True, index = False)